### Characteristics of a Drone

Adapted from: D. Shi, X. Dai, X. Zhang, and Q. Quan, “A practical performance evaluation method for electric multicopters,” IEEE/ASME Transactions on Mechatronics, vol. 22, no. 3, pp. 1337–1348, 2017, https://flyeval.com/js/2017ShiAPracticalPerf.pdf

In [ ]:
# Imports
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# General constants
c_t = 0                       # Thrust coefficient
c_m = 0                       # Torque coefficient
t = 18                        # Temperature (C)
h = 100                       # Altitude
p_0 = 101325                  # Sea level atmospheric pressure (Pa)
q_0 = 1.225                   # Sea level air density at 15C (kg/m^3)
drone_weight = 15             # Drone weight (kg)

# Propeller
pitch_prop = 9                # Propeller pitch (inch)
diam_prop = 28                # Propeller diameter (inch)
blade_number = 2
nr_prop = 6                   # Number of props
weigh_prop = 0.1578           # Propeller weight (kg)

# Motor
no_load_constant = 150        # No load KV_0
max_current_motor = 70        # Maximum current (A)
no_load_current = 1           # No load current (A)
no_load_voltage = 52.2        # No load voltage (V)
resistance_motor = 0.13       # Resistance (ohm)
weight_motor = 0.43           # Motor weight (kg)

# ESC
max_current_esc = 60          # Maximum current (A)
resistance_esc = 0.008        # Resistance (ohm)
weight_esc = 0.073            # Motor weight (kg)

# Battery
capacity = 31000              # Capacity
discharge = 40                # Maximum discharge rate (Kb)
voltage_battery = 44.4        # Total voltage (V)
resistance_battery = 0.004    # Resistance (ohm)
weight_battery = 6.668        # Motor weight (kg)

In [ ]:
p =  p_0 * pow((1 - 0.0065 * h / (273 + t)), 5.2561)   # pressure (Pa)
q = (273 * p / (101325 * (273 + t))) * q_0             # air density (kg/m^3)

In [ ]:
# Calculate coefficients
A = 5           # aspect ratio
ε = 0.85        # correction factor that arises due to downwash
λ = 0.75        # correction coefficient
ζ = 0.5
e = 0.83        # Oswald factor
cf_d = 0.015    # zero-lift drag coefficient
a_0 = - math.pi / 36
K_0 = 6.11

def get_lift_drag(speed):
  # Torque and Thrust coeficients
  c_d = cf_d + (math.pi * A * pow(K_0, 2) / e) * pow(math.atan(pitch_prop /(math.pi * diam_prop)) - a_0, 2)/pow((math.pi * A + K_0), 2)

  c_t = 0.25 * pow(math.pi, 3) * λ * pow (ζ, 2) * blade_number * K_0 * ((math.atan(pitch_prop /(math.pi * diam_prop))) * ε - a_0) / (math.pi * A + K_0)
  c_m = (1 / (8 * A)) * pow(math.pi, 2) * c_d * pow (ζ, 2) * λ * pow(blade_number, 2)
  return c_t, c_m


def propeller_model(speed):
  current_t, current_m = get_lift_drag(speed)

  t = 5.482 * pow(10, -4) * q * pow((speed / 60), 2) * diam_prop  # Thrust (N)
  m = 1.969 * pow(10, -5) * q * pow((speed / 60), 2) * diam_prop  # Torque (Nm)
  return t,m


def motor_model(speed, torque):
  u_motor = resistance_motor * ((torque * no_load_constant * no_load_voltage / (9.55 * (no_load_voltage - no_load_current * resistance_motor)))
            + no_load_current) + (no_load_voltage - no_load_current * resistance_motor) * speed / (no_load_constant * no_load_voltage)
  i_motor = torque * no_load_constant * no_load_voltage / (9.55 * (no_load_voltage - no_load_current * resistance_motor)) + no_load_current
  return u_motor, i_motor


def esc_model(u_motor, i_motor):
  duty_cycle = (u_motor + i_motor * resistance_esc) / voltage_battery
  i_esc = duty_cycle * i_motor
  i_control = 2 # amps for electronics
  i_battery = nr_prop * (i_esc + i_motor + i_control)
  u_esc = voltage_battery - i_battery * resistance_battery - i_motor * resistance_motor
  return u_esc, i_esc, i_battery


def battery_model(i_battery):
  hover_time = ( ( capacity - capacity * 0.15 ) / i_battery) * (60 / 1000)
  return hover_time

In [ ]:
T = []
M = []
U_motor = []
I_motor = []
U_esc = []
I_esc = []
Hover_time = []

speeds = range(2000, 5001, 500)

for speed in speeds:
  # Propeller
  t, m = propeller_model(speed)
  T.append(t)
  M.append(m)

  # Motor
  u_motor, i_motor = motor_model(speed, m)
  U_motor.append(u_motor)
  I_motor.append(i_motor)

  # ESC
  u_esc, i_esc, i_battery = esc_model(u_motor, i_motor)
  I_esc.append(i_esc)
  U_esc.append(u_esc)

  # Battery modelling
  hover_time = ( ( capacity - capacity * 0.15 ) / i_battery) * (60 / 1000)
  Hover_time.append(hover_time)

In [ ]:
# Plotting Propeller Characteristics
fig, (ax1, ax2) = plt.subplots(2)
fig.suptitle('Propellers Thrust and Torque')

ax1.plot(speeds, T)
ax2.plot(speeds, M)

ax1.grid(linestyle = "dashed")
ax2.grid(linestyle = "dashed")

ax1.set(xlabel=' ', ylabel='Thurst (N)')
ax2.set(xlabel='Speed (r/min)', ylabel='Torque (Nm)')

plt.show()

In [ ]:
# Plotting Motor Characteristics
fig, (ax1, ax2) = plt.subplots(2)
fig.suptitle('Motor characteristics')

ax1.plot(speeds, U_motor)
ax2.plot(speeds, I_motor)

ax1.grid(linestyle = "dashed")
ax2.grid(linestyle = "dashed")

ax1.set(xlabel=' ', ylabel='Voltage motor (V)')
ax2.set(xlabel='Speed (r/min)', ylabel='Current motor (A)')

plt.show()

In [ ]:
# Plotting ESC Characteristics
fig, (ax1, ax2) = plt.subplots(2)
fig.suptitle('ESC characteristics')

ax1.plot(speeds, I_esc)
ax1.grid(linestyle = "dashed")
ax1.set(xlabel='Speed (r/min)', ylabel='Current ESC (A)')

plt.show()

In [ ]:
# Plotting Hover vs Speeds Characteristics
plt.plot(speeds, Hover_time)
plt.title("Hover time vs speed")
plt.xlabel("Speed (r/min)")
plt.ylabel("Hover time (min)")
plt.grid(linestyle = "dashed")
plt.show()

In [ ]:
# Drone Performance based on capacity vs hover time

# Test data generated from https://flyeval.com/
hover_time = [3.64, 5.09, 6.55, 8.00, 9.46, 10.91, 12.37, 13.82, 15.28, 16.74, 18.19, 19.65, 21.1, 22.56] # min
range = [2.05, 2.86, 3.68, 4.5, 5.32, 6.14, 6.96, 7.77, 8.59, 9.41, 10.23, 11.05, 11.87, 12.68] # km
capacity = np.linspace(5000, 31001, 14) #mah

fig, ax1 = plt.subplots()
ax1.set_title('Drone Performance based on capacity vs hover time')

ax1.plot(capacity, hover_time, lw=2)
ax1.set_ylabel("Hover time (min)")
ax1.set_xlabel("Capacity (mAh)")

ax2 = ax1.twinx()
ax2.plot(capacity, range, alpha=0)
ax2.set_ylabel("Range (km)")